## Capstone Project - The Battle of Neighborhoods

## Introduction

X is a software company and their customer company Y is based in Toronto organizing event. Company X and Y deal with a project and currently work on a this project to organize an event for 5 days for a group of developers from all over the world.
Company Y has determine to a program including, hotels, hall of meetings, places for visiting, places for shopping, restaurants, cafes etc.
Company Y has to show the list of places in Toronto to attendees, including cafes, restaurants, hall of meetings, hotels etc. It has provide a map in case attendee can't use here mobile application for any reason.

## Data

 The data used in this project is provided by Forsquare location data. The data grouped by landscape area, and each included the information about this area and all information about restaurants, cafes, and stores in which this area.

## Table of contents

#### 01- Import Libraries
#### 02- Define Foursquare Credentials
#### 03- Define the city and get its latitude & longitude
#### 04- Search for Hotels & clean dataframe
#### 05- Search for Parks & clean dataframe
#### 06- Search for Restaurants & clean dataframe
#### 07- Search for Cafeteria & clean dataframe
#### 08- Search for Shopping Stores & clean dataframe
#### 09- Generate map to visualize hotels, shopping stores and Cafeteria and how they cluster together¶
#### 10- Generate map to visualize Park, Restaurant and Cafeteria and how they cluster together

### Import Libraries

In [11]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


## Define Foursquare Credentials

In [12]:
ClIENT_ID = 'U5OFVDXO0SD2K0JGDDW0FQR4AIOAOPESXICXTE0GCBS0MTLQ' # your Foursquare ID
ClIENT_SECRET = 'OP1Q1A3KRD4G5OUVBCUC3USXE2PJQHXILXALTMBHJCLCNGJ3' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: U5OFVDXO0SD2K0JGDDW0FQR4AIOAOPESXICXTE0GCBS0MTLQ
Foursquare_Secret:OP1Q1A3KRD4G5OUVBCUC3USXE2PJQHXILXALTMBHJCLCNGJ3


## Define the city and get its latitude & longitude

In [13]:
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


## Search for Hotels

In [14]:
search_query = 'Hotel'
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=U5OFVDXO0SD2K0JGDDW0FQR4AIOAOPESXICXTE0GCBS0MTLQ&client_secret=OP1Q1A3KRD4G5OUVBCUC3USXE2PJQHXILXALTMBHJCLCNGJ3&ll=43.653963,-79.387207&v=20180604&query=Hotel&radius=500&limit=30'

In [15]:
results = requests.get(url).json()

In [16]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ab2d511f964a5209b6c20e3,123 Queen Street West,CA,Toronto,Canada,at York St.,416,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.65112928325278...",43.651129,-79.383829,NaN,M5H 2M9,ON,Sheraton Centre Toronto Hotel,v-1560765644,NaN
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,58b7d72dcc05d161570bd712,123 Queen Street West,CA,Toronto,Canada,NaN,410,"[123 Queen Street West, Toronto ON M5H 2M9, Ca...","[{'label': 'display', 'lat': 43.65101646682632...",43.651016,-79.384148,NaN,M5H 2M9,ON,Sheraton Centre Toronto Hotel,v-1560765644,402127087
2,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",False,4b68aed1f964a520de862be3,194 Queen St W,CA,Toronto,Canada,Queen & St. Patrick,400,"[194 Queen St W (Queen & St. Patrick), Toronto...","[{'label': 'display', 'lat': 43.65050475544005...",43.650505,-79.388577,NaN,M5V 1Z1,ON,The Rex Hotel Jazz & Blues Bar,v-1560765644,62225795
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,52ce14b0498e50457ce11780,108 Chestnut Street,CA,Toronto,Canada,Dundas St W,124,"[108 Chestnut Street (Dundas St W), Toronto ON...","[{'label': 'display', 'lat': 43.6546083, 'lng'...",43.654608,-79.385942,NaN,M5G 1R3,ON,DoubleTree by Hilton Hotel Toronto Downtown,v-1560765644,NaN
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4f343a31e4b0230a3b337a90,123 Test Drive,CA,Toronto,Canada,at somewhere St,500,"[123 Test Drive (at somewhere St), Toronto ON ...","[{'label': 'display', 'lat': 43.658434, 'lng':...",43.658434,-79.387894,NaN,M2M 2M2,ON,VFM Test Hotel,v-1560765644,NaN


## Clean Hotel Dataframe

In [17]:
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,CA,Toronto,Canada,at York St.,416,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.65112928325278...",43.651129,-79.383829,NaN,M5H 2M9,ON,4ab2d511f964a5209b6c20e3
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,CA,Toronto,Canada,NaN,410,"[123 Queen Street West, Toronto ON M5H 2M9, Ca...","[{'label': 'display', 'lat': 43.65101646682632...",43.651016,-79.384148,NaN,M5H 2M9,ON,58b7d72dcc05d161570bd712
2,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,CA,Toronto,Canada,Queen & St. Patrick,400,"[194 Queen St W (Queen & St. Patrick), Toronto...","[{'label': 'display', 'lat': 43.65050475544005...",43.650505,-79.388577,NaN,M5V 1Z1,ON,4b68aed1f964a520de862be3
3,DoubleTree by Hilton Hotel Toronto Downtown,Hotel,108 Chestnut Street,CA,Toronto,Canada,Dundas St W,124,"[108 Chestnut Street (Dundas St W), Toronto ON...","[{'label': 'display', 'lat': 43.6546083, 'lng'...",43.654608,-79.385942,NaN,M5G 1R3,ON,52ce14b0498e50457ce11780
4,VFM Test Hotel,Hotel,123 Test Drive,CA,Toronto,Canada,at somewhere St,500,"[123 Test Drive (at somewhere St), Toronto ON ...","[{'label': 'display', 'lat': 43.658434, 'lng':...",43.658434,-79.387894,NaN,M2M 2M2,ON,4f343a31e4b0230a3b337a90


In [18]:
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651129,-79.383829,M5H 2M9,ON
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651016,-79.384148,M5H 2M9,ON
2,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
3,DoubleTree by Hilton Hotel Toronto Downtown,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
4,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
5,Sheraton Centre Toronto Hotel - Grand Ballroom,Ballroom,123 Queen Street West,43.651200,-79.384520,NaN,ON
6,Stathcona Hotel,Hotel,NaN,43.654947,-79.386359,NaN,ON
7,"hilton hotel, richmondstreet E, 145, toronto",Hotel Pool,"Richmondstreet E, 145",43.650625,-79.385488,NaN,ON
8,Be SixFifty Hotel,Hotel,650 Bay Street,43.657120,-79.384560,NaN,ON
9,650 Hotel,Hotel,650 Bay Street,43.657046,-79.384411,NaN,ON


In [19]:
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651129,-79.383829,M5H 2M9,ON
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651016,-79.384148,M5H 2M9,ON
2,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
3,DoubleTree by Hilton Hotel Toronto Downtown,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
4,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
10,Hilton Toronto,Hotel,145 Richmond St W,43.650143,-79.385488,M5H 2L2,ON
11,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
12,Fitness Centre,Gym,Sheraton Centre,43.650985,-79.384002,M5H 2M9,ON
14,89 Chestnut Residence,College Residence Hall,89 Chestnut St.,43.654160,-79.385291,M5G 1R1,ON
15,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON


In [20]:
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651129,-79.383829,M5H 2M9,ON
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651016,-79.384148,M5H 2M9,ON
3,DoubleTree by Hilton Hotel Toronto Downtown,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
4,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
10,Hilton Toronto,Hotel,145 Richmond St W,43.650143,-79.385488,M5H 2L2,ON
11,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
15,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON
18,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


In [21]:
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651129,-79.383829,M5H 2M9,ON
3,DoubleTree by Hilton Hotel Toronto Downtown,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
4,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
10,Hilton Toronto,Hotel,145 Richmond St W,43.650143,-79.385488,M5H 2L2,ON
11,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
15,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON
18,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


In [22]:
df_hotel = df_hotels[df_hotels.postalCode == 'M5H 2M9']
df_hotel

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651129,-79.383829,M5H 2M9,ON
18,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


## Search for Parks

In [23]:
search_query = 'Park'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=U5OFVDXO0SD2K0JGDDW0FQR4AIOAOPESXICXTE0GCBS0MTLQ&client_secret=OP1Q1A3KRD4G5OUVBCUC3USXE2PJQHXILXALTMBHJCLCNGJ3&ll=43.653963,-79.387207&v=20180604&query=Park&radius=10000&limit=30'

In [24]:
presults = requests.get(url).json()

In [25]:
venues = presults['response']['venues']

park_dataframe = json_normalize(venues)
park_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,4b170e47f964a5208cc123e3,691 University Ave,CA,Toronto,Canada,at College St,716,"[691 University Ave (at College St), Toronto O...","[{'label': 'display', 'lat': 43.66000635700919...",43.660006,-79.390272,M5G 2P1,ON,Queen's Park Subway Station,v-1560766125,NaN
1,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4b54deadf964a520a3d027e3,NaN,CA,Toronto,Canada,NaN,423,"[btwn College, University, Queen & Spadina, To...","[{'label': 'display', 'lat': 43.65248755299856...",43.652488,-79.392053,NaN,ON,Grange Park,v-1560766125,NaN
2,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4ad4c05ef964a520bdf620e3,1053 Dundas St. W.,CA,Toronto,Canada,at Gore Vale Ave.,2271,"[1053 Dundas St. W. (at Gore Vale Ave.), Toron...","[{'label': 'display', 'lat': 43.64707192840234...",43.647072,-79.413756,M5H 2N2,ON,Trinity Bellwoods Park,v-1560766125,NaN
3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4bb3c91a2397b7134d9f38b3,Spadina Ave,CA,Toronto,Canada,at Wellington St W,1223,"[Spadina Ave (at Wellington St W), Toronto ON,...","[{'label': 'display', 'lat': 43.64422227794523...",43.644222,-79.394230,NaN,ON,Clarence Square Park,v-1560766125,NaN
4,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,4ad80fcef964a520a61021e3,280 Simcoe St,CA,Toronto,Canada,Dundas Street West,282,"[280 Simcoe St (Dundas Street West), Toronto O...","[{'label': 'display', 'lat': 43.65544241747622...",43.655442,-79.390056,M5T 2Y5,ON,One Park Lane,v-1560766125,NaN


## Clean Park Dataframe

In [26]:
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Queen's Park Subway Station,Metro Station,691 University Ave,CA,Toronto,Canada,at College St,716,"[691 University Ave (at College St), Toronto O...","[{'label': 'display', 'lat': 43.66000635700919...",43.660006,-79.390272,M5G 2P1,ON,4b170e47f964a5208cc123e3
1,Grange Park,Park,NaN,CA,Toronto,Canada,NaN,423,"[btwn College, University, Queen & Spadina, To...","[{'label': 'display', 'lat': 43.65248755299856...",43.652488,-79.392053,NaN,ON,4b54deadf964a520a3d027e3
2,Trinity Bellwoods Park,Park,1053 Dundas St. W.,CA,Toronto,Canada,at Gore Vale Ave.,2271,"[1053 Dundas St. W. (at Gore Vale Ave.), Toron...","[{'label': 'display', 'lat': 43.64707192840234...",43.647072,-79.413756,M5H 2N2,ON,4ad4c05ef964a520bdf620e3
3,Clarence Square Park,Park,Spadina Ave,CA,Toronto,Canada,at Wellington St W,1223,"[Spadina Ave (at Wellington St W), Toronto ON,...","[{'label': 'display', 'lat': 43.64422227794523...",43.644222,-79.394230,NaN,ON,4bb3c91a2397b7134d9f38b3
4,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,CA,Toronto,Canada,Dundas Street West,282,"[280 Simcoe St (Dundas Street West), Toronto O...","[{'label': 'display', 'lat': 43.65544241747622...",43.655442,-79.390056,M5T 2Y5,ON,4ad80fcef964a520a61021e3


In [27]:
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
1,Grange Park,Park,NaN,43.652488,-79.392053,NaN,ON
2,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
3,Clarence Square Park,Park,Spadina Ave,43.644222,-79.394230,NaN,ON
4,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
5,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
6,St. Andrews Playground / Dog Park,Park,450 Adelaide Street West,43.647388,-79.398752,NaN,ON
7,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
8,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
9,Berczy Park,Park,35 Wellington St. East,43.648048,-79.375172,NaN,ON


In [28]:
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
2,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
4,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
5,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
7,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
8,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
10,GoodLife Fitness Toronto Bloor Park,Gym,8 Park Rd,43.671398,-79.384408,M4W 3G8,ON
12,High Park,Park,1873 Bloor St. W,43.646479,-79.463425,M6R 2Z3,ON
13,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
18,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON


In [29]:
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park

,name,categories,address,lat,lng,postalCode,state
2,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
7,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
8,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
12,High Park,Park,1873 Bloor St. W,43.646479,-79.463425,M6R 2Z3,ON
13,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
19,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
24,Coronation Park,Park,711 Lake Shore Blvd. W,43.634553,-79.405403,M5V 3T7,ON
27,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,M5V 3Z1,ON
29,Barbara Hall Park,Park,519 Church St,43.666879,-79.381068,M4Y 2K9,ON


## Search for Restaurants

In [30]:
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=U5OFVDXO0SD2K0JGDDW0FQR4AIOAOPESXICXTE0GCBS0MTLQ&client_secret=OP1Q1A3KRD4G5OUVBCUC3USXE2PJQHXILXALTMBHJCLCNGJ3&ll=43.653963,-79.387207&v=20180604&query=Restaurant&radius=10000&limit=30'

In [31]:
Rresults = requests.get(url).json()

In [32]:
venues = Rresults['response']['venues']

Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",False,4ad4c05ff964a52048f720e3,110 Chestnut Street,CA,Toronto,Canada,NaN,145,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,NaN,M5G 1R3,ON,Hemispheres Restaurant & Bistro,v-1560769870,NaN
1,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",False,4ad4c060f964a5207ff720e3,323 Spadina Ave.,CA,Toronto,Canada,at D'Arcy St.,922,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...","[{'label': 'display', 'lat': 43.65431754076345...",43.654318,-79.398650,Kensington Market,M5T 2E9,ON,Rol San Restaurant 龍笙棧,v-1560769870,NaN
2,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4b223f5af964a520ba4424e3,225 Frnt St W,CA,Toronto,Canada,NaN,1039,"[225 Frnt St W, Toronto ON M5V 2X3, Canada]","[{'label': 'display', 'lat': 43.64474919591934...",43.644749,-79.385113,Entertainment District,M5V 2X3,ON,Azure Restaurant & Bar,v-1560769870,136175835
3,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4ad4c05cf964a52006f620e3,37 King Street East,CA,Toronto,Canada,at Le Meridien King Edward Hotel,1011,[37 King Street East (at Le Meridien King Edwa...,"[{'label': 'display', 'lat': 43.64929834396347...",43.649298,-79.376431,NaN,M5C 1E9,ON,Victoria's Restaurant,v-1560769870,498556908
4,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",False,4b072e9df964a52009f922e3,280 Spadina Ave.,CA,Toronto,Canada,at Dundas St. W.,892,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...","[{'label': 'display', 'lat': 43.65278331265585...",43.652783,-79.398174,NaN,NaN,ON,Sky Dragon Chinese Restaurant 龍翔酒樓,v-1560769870,NaN


## Clean Restaurant Dataframe

In [33]:
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,CA,Toronto,Canada,NaN,145,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,NaN,M5G 1R3,ON,4ad4c05ff964a52048f720e3
1,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,CA,Toronto,Canada,at D'Arcy St.,922,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...","[{'label': 'display', 'lat': 43.65431754076345...",43.654318,-79.398650,Kensington Market,M5T 2E9,ON,4ad4c060f964a5207ff720e3
2,Azure Restaurant & Bar,Restaurant,225 Frnt St W,CA,Toronto,Canada,NaN,1039,"[225 Frnt St W, Toronto ON M5V 2X3, Canada]","[{'label': 'display', 'lat': 43.64474919591934...",43.644749,-79.385113,Entertainment District,M5V 2X3,ON,4b223f5af964a520ba4424e3
3,Victoria's Restaurant,Restaurant,37 King Street East,CA,Toronto,Canada,at Le Meridien King Edward Hotel,1011,[37 King Street East (at Le Meridien King Edwa...,"[{'label': 'display', 'lat': 43.64929834396347...",43.649298,-79.376431,NaN,M5C 1E9,ON,4ad4c05cf964a52006f620e3
4,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,CA,Toronto,Canada,at Dundas St. W.,892,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...","[{'label': 'display', 'lat': 43.65278331265585...",43.652783,-79.398174,NaN,NaN,ON,4b072e9df964a52009f922e3


In [34]:
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
2,Azure Restaurant & Bar,Restaurant,225 Frnt St W,43.644749,-79.385113,M5V 2X3,ON
3,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
4,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,43.652783,-79.398174,NaN,ON
5,360 Restaurant,Wine Bar,301 Front St W,43.642537,-79.387042,M5V 2T6,ON
6,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON
7,Victor Restaurant & Bar,Bar,30 Mercer Street,43.645634,-79.391125,M5V 1H3,ON
8,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,M5T 2E6,ON
9,Green Tea Restaurant Downtown,Chinese Restaurant,261 Spadina Avenue. Upper level,43.652488,-79.397501,M5T 2E3,ON


In [35]:
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
2,Azure Restaurant & Bar,Restaurant,225 Frnt St W,43.644749,-79.385113,M5V 2X3,ON
3,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
5,360 Restaurant,Wine Bar,301 Front St W,43.642537,-79.387042,M5V 2T6,ON
6,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON
7,Victor Restaurant & Bar,Bar,30 Mercer Street,43.645634,-79.391125,M5V 1H3,ON
8,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,M5T 2E6,ON
9,Green Tea Restaurant Downtown,Chinese Restaurant,261 Spadina Avenue. Upper level,43.652488,-79.397501,M5T 2E3,ON
10,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,M5T 2G3,ON


## Search for Cafeteria

In [36]:
search_query = 'Cafeteria'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=U5OFVDXO0SD2K0JGDDW0FQR4AIOAOPESXICXTE0GCBS0MTLQ&client_secret=OP1Q1A3KRD4G5OUVBCUC3USXE2PJQHXILXALTMBHJCLCNGJ3&ll=43.653963,-79.387207&v=20180604&query=Cafeteria&radius=10000&limit=30'

In [37]:
cresults = requests.get(url).json()

In [38]:
venues = cresults['response']['venues']

Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,4f9165bde4b04fef0087e4e2,The Hospital for Sick Children (SickKids),CA,Toronto,Canada,NaN,372,"[The Hospital for Sick Children (SickKids), To...","[{'label': 'display', 'lat': 43.65720930549997...",43.657209,-79.386063,NaN,ON,Cafeteria,v-1560770008
1,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",False,528e99a211d262edc3708a6f,388 Spadina Ave,CA,Toronto,Canada,at Nassau St,977,"[388 Spadina Ave (at Nassau St), Toronto ON M5...","[{'label': 'display', 'lat': 43.65545427446646...",43.655454,-79.399163,M5T 2G5,ON,Stay Cafeteria 慢走,v-1560770008
2,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",False,56b2c90a498e9f0e73d6bed6,974 College St,CA,Toronto,Canada,NaN,3260,"[974 College St, Toronto ON M6H 1A5, Canada]","[{'label': 'display', 'lat': 43.65341883567197...",43.653419,-79.427678,M6H 1A5,ON,Cafeteria,v-1560770008
3,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",False,4ec2b68a8231a83de8c45191,100 McCaul St.,CA,Toronto,Canada,Second Floor,389,"[100 McCaul St. (Second Floor), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65278016892648...",43.652780,-79.391762,NaN,ON,OCAD Cafeteria,v-1560770008
4,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",False,53bd6dfe498e6c1d3bc9333e,NaN,CA,Toronto,Canada,NaN,260,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655657, 'lng':...",43.655657,-79.389443,NaN,ON,Industrial Alliance Cafeteria,v-1560770008


## Clean Cafeteria Dataframe


In [39]:
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),CA,Toronto,Canada,NaN,372,"[The Hospital for Sick Children (SickKids), To...","[{'label': 'display', 'lat': 43.65720930549997...",43.657209,-79.386063,NaN,ON,4f9165bde4b04fef0087e4e2
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,CA,Toronto,Canada,at Nassau St,977,"[388 Spadina Ave (at Nassau St), Toronto ON M5...","[{'label': 'display', 'lat': 43.65545427446646...",43.655454,-79.399163,M5T 2G5,ON,528e99a211d262edc3708a6f
2,Cafeteria,Mexican Restaurant,974 College St,CA,Toronto,Canada,NaN,3260,"[974 College St, Toronto ON M6H 1A5, Canada]","[{'label': 'display', 'lat': 43.65341883567197...",43.653419,-79.427678,M6H 1A5,ON,56b2c90a498e9f0e73d6bed6
3,OCAD Cafeteria,College Cafeteria,100 McCaul St.,CA,Toronto,Canada,Second Floor,389,"[100 McCaul St. (Second Floor), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65278016892648...",43.652780,-79.391762,NaN,ON,4ec2b68a8231a83de8c45191
4,Industrial Alliance Cafeteria,Cafeteria,NaN,CA,Toronto,Canada,NaN,260,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655657, 'lng':...",43.655657,-79.389443,NaN,ON,53bd6dfe498e6c1d3bc9333e


In [40]:
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,NaN,ON
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
2,Cafeteria,Mexican Restaurant,974 College St,43.653419,-79.427678,M6H 1A5,ON
3,OCAD Cafeteria,College Cafeteria,100 McCaul St.,43.652780,-79.391762,NaN,ON
4,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,NaN,ON
5,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,NaN,NaN
6,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,NaN,ON
7,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,NaN,ON
8,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
9,Med Sci Cafeteria,College Cafeteria,NaN,43.660260,-79.392977,NaN,ON


In [41]:
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria

,name,categories,address,lat,lng,postalCode,state
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
2,Cafeteria,Mexican Restaurant,974 College St,43.653419,-79.427678,M6H 1A5,ON
8,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
21,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,M5B 1W8,ON
22,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON
24,George Brown College St. James Campus,Community College,51 Dockside Dr,43.644163,-79.366375,M5A 0B6,ON
29,One Hour,Chinese Restaurant,435 Spadina Ave.,43.657399,-79.399649,M5T 2T6,ON


## Search for Shopping Stores

In [42]:
search_query = 'Shopping'
radius = 1000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=U5OFVDXO0SD2K0JGDDW0FQR4AIOAOPESXICXTE0GCBS0MTLQ&client_secret=OP1Q1A3KRD4G5OUVBCUC3USXE2PJQHXILXALTMBHJCLCNGJ3&ll=43.653963,-79.387207&v=20180604&query=Shopping&radius=1000&limit=30'

In [43]:
sresults = requests.get(url).json()

In [44]:
venues = sresults['response']['venues']

Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4ad4c062f964a520fcf720e3,280 Spadina Ave,CA,Toronto,Canada,at Dundas St W,897,"[280 Spadina Ave (at Dundas St W), Toronto ON ...","[{'label': 'display', 'lat': 43.65277367198862...",43.652774,-79.398222,NaN,M5T 3A5,ON,Dragon City Shopping Mall 龍城,v-1560770335
1,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4b9e86dff964a520e6eb36e3,66 Wellington St W,CA,Toronto,Canada,btw York & Bay St,908,"[66 Wellington St W (btw York & Bay St), Toron...","[{'label': 'display', 'lat': 43.64718424611984...",43.647184,-79.380932,NaN,M5K 1A1,ON,TD Centre Shopping Concourse,v-1560770335
2,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4ad77a12f964a520260b21e3,220 Yonge St,CA,Toronto,Canada,btwn Queen & Dundas,527,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,43.653852,-79.380655,"Downtown Toronto, Toronto, ON",M5B 2H1,ON,CF Toronto Eaton Centre,v-1560770335
3,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",False,5840cafe06f1a34af80cc609,176 Yonge Street,CA,Toronto,Canada,Queen Street West,688,"[176 Yonge Street (Queen Street West), Toronto...","[{'label': 'display', 'lat': 43.65181, 'lng': ...",43.651810,-79.379192,NaN,M5C 2L7,ON,Saks Fifth Avenue Club - Personal Shopping,v-1560770335
4,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4b83d653f964a520d91231e3,461 King St. West,CA,Toronto,Canada,NaN,974,"[461 King St. West, Toronto ON M5V 1K4, Canada]","[{'label': 'display', 'lat': 43.64795765580337...",43.647958,-79.396006,NaN,M5V 1K4,ON,Yellow Pages - RedFlagDeals.com,v-1560770335


## Clean Shopping Dataframe

In [45]:
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,CA,Toronto,Canada,at Dundas St W,897,"[280 Spadina Ave (at Dundas St W), Toronto ON ...","[{'label': 'display', 'lat': 43.65277367198862...",43.652774,-79.398222,NaN,M5T 3A5,ON,4ad4c062f964a520fcf720e3
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,CA,Toronto,Canada,btw York & Bay St,908,"[66 Wellington St W (btw York & Bay St), Toron...","[{'label': 'display', 'lat': 43.64718424611984...",43.647184,-79.380932,NaN,M5K 1A1,ON,4b9e86dff964a520e6eb36e3
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,CA,Toronto,Canada,btwn Queen & Dundas,527,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,43.653852,-79.380655,"Downtown Toronto, Toronto, ON",M5B 2H1,ON,4ad77a12f964a520260b21e3
3,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,CA,Toronto,Canada,Queen Street West,688,"[176 Yonge Street (Queen Street West), Toronto...","[{'label': 'display', 'lat': 43.65181, 'lng': ...",43.651810,-79.379192,NaN,M5C 2L7,ON,5840cafe06f1a34af80cc609
4,Yellow Pages - RedFlagDeals.com,Office,461 King St. West,CA,Toronto,Canada,NaN,974,"[461 King St. West, Toronto ON M5V 1K4, Canada]","[{'label': 'display', 'lat': 43.64795765580337...",43.647958,-79.396006,NaN,M5V 1K4,ON,4b83d653f964a520d91231e3


In [46]:
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,43.652774,-79.398222,M5T 3A5,ON
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.653852,-79.380655,M5B 2H1,ON
3,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
4,Yellow Pages - RedFlagDeals.com,Office,461 King St. West,43.647958,-79.396006,M5V 1K4,ON


In [47]:
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping

,name,categories,address,lat,lng,postalCode,state
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,43.652774,-79.398222,M5T 3A5,ON
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.653852,-79.380655,M5B 2H1,ON


## Generate map to visualize hotels, shopping stores and Cafeteria and how they cluster together

In [48]:
hotel_neighbourhood_df = pd.concat([df_hotel, df_Cafeteria, df_Shopping], ignore_index=True)
hotel_neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651129,-79.383829,M5H 2M9,ON
1,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON
2,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
3,Cafeteria,Mexican Restaurant,974 College St,43.653419,-79.427678,M6H 1A5,ON
4,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
5,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,M5B 1W8,ON
6,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON
7,George Brown College St. James Campus,Community College,51 Dockside Dr,43.644163,-79.366375,M5A 0B6,ON
8,One Hour,Chinese Restaurant,435 Spadina Ave.,43.657399,-79.399649,M5T 2T6,ON
9,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,43.652774,-79.398222,M5T 3A5,ON


In [49]:
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

## Generate map to visualize Park, Restaurant and Cafeteria and how they cluster together

In [50]:
park_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
park_neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
1,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
2,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
3,High Park,Park,1873 Bloor St. W,43.646479,-79.463425,M6R 2Z3,ON
4,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
5,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
6,Coronation Park,Park,711 Lake Shore Blvd. W,43.634553,-79.405403,M5V 3T7,ON
7,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,M5V 3Z1,ON
8,Barbara Hall Park,Park,519 Church St,43.666879,-79.381068,M4Y 2K9,ON
9,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON


In [51]:
park_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(park_map)  
    
park_map